## Embedding Chunked Data

## Tourism Data Embedding with all-MiniLM-L6-v2

**Optimized for GTX 1650 Max-Q** - Generates 384-dimensional embeddings for 4,160+ tourism chunks.

**Key Features:**
- ✅ Memory-optimized batch processing (16 chunks/batch)
- ✅ Content enrichment with location and category metadata  
- ✅ GPU memory management with periodic cache clearing
- ✅ Multiple output formats (JSON + NumPy arrays)
- ✅ Built-in quality testing with similarity search

In [1]:
import json
import numpy as np
import torch
import time
from pathlib import Path
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

/home/karan/miniconda3/envs/knji/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load and initialize the embedding model
print("🔄 Loading all-MiniLM-L6-v2 model...")
model = SentenceTransformer('all-MiniLM-L6-v2')

print(f"✅ Model loaded successfully!")
print(f"📏 Model output dimensions: {model.get_sentence_embedding_dimension()}")
print(f"🚀 Model max sequence length: {model.max_seq_length}")

🔄 Loading all-MiniLM-L6-v2 model...
✅ Model loaded successfully!
📏 Model output dimensions: 384
🚀 Model max sequence length: 256


In [2]:
# Load the optimized tourism chunks
def load_tourism_chunks(chunks_file: str = "../optimized_chunks/all_chunks_optimized.json"):
    """Load the chunked tourism data for embedding"""
    print(f"📂 Loading chunks from {chunks_file}...")
    
    with open(chunks_file, 'r', encoding='utf-8') as f:
        chunks = json.load(f)
    
    print(f"✅ Loaded {len(chunks):,} tourism chunks")
    
    # Display sample chunk info
    sample_chunk = chunks[0]
    print(f"\n📊 Sample chunk analysis:")
    print(f"   Content length: {len(sample_chunk['content'])} characters")
    print(f"   Location: {sample_chunk['location']['city']}, {sample_chunk['location']['state']}")
    print(f"   Category: {sample_chunk['classification']['category']} → {sample_chunk['classification']['subcategory']}")
    print(f"   Content preview: {sample_chunk['content'][:100]}...")
    
    return chunks

# Load the chunks
tourism_chunks = load_tourism_chunks()

📂 Loading chunks from ../optimized_chunks/all_chunks_optimized.json...
✅ Loaded 4,160 tourism chunks

📊 Sample chunk analysis:
   Content length: 150 characters
   Location: India, India
   Category: general → general
   Content preview: India (Hindi: भारत or Bhārat), the largest country in South Asia, has many of the world's highest mo...


In [4]:
# Prepare content for embedding
def prepare_embedding_content(chunks):
    """Prepare optimized content for embedding generation"""
    embedding_data = []
    
    for chunk in chunks:
        # Create rich content for embedding by combining content with metadata
        content = chunk['content']
        
        # Add location context for better geographic understanding
        location_context = f"{chunk['location']['city']}, {chunk['location']['state']}"
        
        # Add category context for better semantic understanding  
        category_context = f"{chunk['classification']['category']} - {chunk['classification']['subcategory']}"
        
        # Combine content with minimal metadata for better embeddings
        enriched_content = f"{content} Location: {location_context}. Category: {category_context}"
        
        embedding_data.append({
            'chunk_id': chunk['chunk_id'],
            'original_content': content,
            'embedding_content': enriched_content,
            'metadata': {
                'location': chunk['location'],
                'classification': chunk['classification'],
                'practical_info': chunk['practical_info'],
                'relevance_scores': chunk['relevance_scores']
            }
        })
    
    return embedding_data

# Prepare embedding content
print("🔄 Preparing content for embedding...")
embedding_data = prepare_embedding_content(tourism_chunks)

print(f"✅ Prepared {len(embedding_data):,} items for embedding")
print(f"📝 Sample enriched content:")
print(f"   Original: {embedding_data[0]['original_content'][:80]}...")
print(f"   Enriched: {embedding_data[0]['embedding_content'][:120]}...")

🔄 Preparing content for embedding...
✅ Prepared 4,160 items for embedding
📝 Sample enriched content:
   Original: India (Hindi: भारत or Bhārat), the largest country in South Asia, has many of th...
   Enriched: India (Hindi: भारत or Bhārat), the largest country in South Asia, has many of the world's highest mountains, most popula...


In [5]:
# Generate embeddings with all-MiniLM-L6-v2 - Optimized for GTX 1650 Max-Q
def generate_embeddings_batch(model, texts, batch_size=16):  # Reduced batch size for GTX 1650
    """Generate embeddings in batches optimized for GTX 1650 Max-Q"""
    all_embeddings = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print(f"🎯 Using device: {device}")
    if device.type == 'cuda':
        print(f"🔧 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    model.to(device)
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
        batch_texts = texts[i:i+batch_size]
        
        # Generate embeddings with memory management
        with torch.no_grad():
            embeddings = model.encode(
                batch_texts, 
                convert_to_tensor=True, 
                device=device,
                show_progress_bar=False  # Disable inner progress bar
            )
            # Move to CPU and convert to numpy to free GPU memory
            embeddings = embeddings.cpu().numpy()
            all_embeddings.extend(embeddings)
        
        # Clear GPU cache periodically for GTX 1650
        if device.type == 'cuda' and i % (batch_size * 10) == 0:
            torch.cuda.empty_cache()
    
    return np.array(all_embeddings)

# Extract texts for embedding
embedding_texts = [item['embedding_content'] for item in embedding_data]

print("🚀 Generating embeddings optimized for GTX 1650 Max-Q...")
print(f"📊 Processing {len(embedding_texts):,} texts")
print(f"🔧 Batch size: 16 (optimized for 4GB VRAM)")
print(f"💾 Expected output dimensions: 384")
print(f"⏱️ Estimated time: ~5-15 minutes")

# Generate embeddings
start_time = time.time()
embeddings = generate_embeddings_batch(model, embedding_texts)
end_time = time.time()

print(f"\n✅ Embedding generation complete!")
print(f"📐 Shape: {embeddings.shape}")
print(f"📊 Dimensions per embedding: {embeddings.shape[1]}")
print(f"💾 Total size: {embeddings.nbytes / 1024 / 1024:.2f} MB")
print(f"⏱️ Processing time: {end_time - start_time:.1f} seconds")
print(f"🚀 Speed: {len(embedding_texts) / (end_time - start_time):.1f} chunks/second")

🚀 Generating embeddings optimized for GTX 1650 Max-Q...
📊 Processing 4,160 texts
🔧 Batch size: 16 (optimized for 4GB VRAM)
💾 Expected output dimensions: 384
⏱️ Estimated time: ~5-15 minutes
🎯 Using device: cuda
🔧 GPU: NVIDIA GeForce GTX 1650
💾 GPU Memory: 3.6 GB


Generating embeddings: 100%|██████████| 260/260 [00:08<00:00, 29.03it/s]


✅ Embedding generation complete!
📐 Shape: (4160, 384)
📊 Dimensions per embedding: 384
💾 Total size: 6.09 MB
⏱️ Processing time: 9.0 seconds
🚀 Speed: 463.9 chunks/second


In [6]:
# Create comprehensive embedding dataset
embedding_dataset = []

for i, (chunk_data, embedding) in enumerate(zip(embedding_data, embeddings)):
    entry = {
        'chunk_id': chunk_data['chunk_id'],
        'content': chunk_data['original_content'],
        'enriched_content': chunk_data['embedding_content'],
        'embedding': embedding.tolist(),  # Convert numpy array to list for JSON serialization
        'metadata': chunk_data['metadata'],
        'embedding_info': {
            'model': 'all-MiniLM-L6-v2',
            'dimensions': len(embedding),
            'norm': float(np.linalg.norm(embedding))
        }
    }
    embedding_dataset.append(entry)

print(f"✅ Created comprehensive embedding dataset")
print(f"📊 Total entries: {len(embedding_dataset):,}")
print(f"📐 Embedding dimensions: {embedding_dataset[0]['embedding_info']['dimensions']}")

# Sample embedding analysis
sample_embedding = embedding_dataset[0]
print(f"\n🔍 Sample embedding analysis:")
print(f"   Chunk ID: {sample_embedding['chunk_id']}")
print(f"   Content: {sample_embedding['content'][:80]}...")
print(f"   Embedding norm: {sample_embedding['embedding_info']['norm']:.4f}")
print(f"   First 5 dimensions: {sample_embedding['embedding'][:5]}")

✅ Created comprehensive embedding dataset
📊 Total entries: 4,160
📐 Embedding dimensions: 384

🔍 Sample embedding analysis:
   Chunk ID: in_in_india_gen_gen_001
   Content: India (Hindi: भारत or Bhārat), the largest country in South Asia, has many of th...
   Embedding norm: 1.0000
   First 5 dimensions: [0.058542847633361816, 0.0032589775510132313, -0.06868673115968704, -0.018095869570970535, -0.008678180165588856]


In [7]:
# Save embeddings to file
import json

# Create embeddings directory if it doesn't exist
embeddings_dir = Path("embeddings")
embeddings_dir.mkdir(exist_ok=True)

# Save the complete embedding dataset
embeddings_file = embeddings_dir / "tourism_embeddings_all_MiniLM_L6_v2.json"

print(f"💾 Saving embedding dataset to: {embeddings_file}")
print(f"📊 Dataset size: {len(embedding_dataset):,} entries")

with open(embeddings_file, 'w', encoding='utf-8') as f:
    json.dump(embedding_dataset, f, indent=2, ensure_ascii=False)

# Calculate file size
file_size_mb = embeddings_file.stat().st_size / 1024 / 1024

print(f"✅ Embeddings saved successfully!")
print(f"📁 File: {embeddings_file}")
print(f"📊 File size: {file_size_mb:.2f} MB")

# Also save just the embedding vectors as numpy array for faster loading
embeddings_numpy_file = embeddings_dir / "tourism_embeddings_vectors.npy"
np.save(embeddings_numpy_file, embeddings)

print(f"🎯 Numpy embeddings saved: {embeddings_numpy_file}")
print(f"📐 Shape: {embeddings.shape}")

💾 Saving embedding dataset to: embeddings/tourism_embeddings_all_MiniLM_L6_v2.json
📊 Dataset size: 4,160 entries
✅ Embeddings saved successfully!
📁 File: embeddings/tourism_embeddings_all_MiniLM_L6_v2.json
📊 File size: 47.43 MB
🎯 Numpy embeddings saved: embeddings/tourism_embeddings_vectors.npy
📐 Shape: (4160, 384)


In [9]:
# Test embedding quality with sample queries
def find_similar_chunks(query, embeddings, embedding_data, model, top_k=5):
    """Find similar chunks using cosine similarity"""
    # Generate embedding for query
    query_embedding = model.encode([query])
    
    # Calculate cosine similarity
    similarities = np.dot(embeddings, query_embedding.T).flatten()
    
    # Get top-k most similar chunks
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    
    results = []
    for idx in top_indices:
        results.append({
            'similarity': float(similarities[idx]),
            'content': embedding_data[idx]['original_content'],
            'metadata': embedding_data[idx]['metadata'],
            'chunk_id': embedding_data[idx]['chunk_id']
        })
    
    return results

# Test with sample tourism queries
test_queries = [
    # "best places to visit in Goa beaches",
    "adventure activities in Himachal Pradesh mountains", 
    # "budget accommodation in Delhi",
    "traditional food in Almora",
    "heritage sites in Uttarakhand"
]

print("🧪 Testing embedding quality with sample queries...")
print(f"🎯 Model: all-MiniLM-L6-v2 (384 dimensions)")
print(f"📊 Testing against {len(embeddings):,} tourism chunks")

for i, query in enumerate(test_queries, 1):
    print(f"\n--- Query {i}: '{query}' ---")
    results = find_similar_chunks(query, embeddings, embedding_data, model, top_k=3)
    
    for j, result in enumerate(results, 1):
        print(f"  {j}. Similarity: {result['similarity']:.4f}")
        print(f"     Content: {result['content'][:100]}...")
        print(f"     Category: {result['metadata']['classification']['category']}")
        print(f"     Location: {result['metadata']['location']['city']}, {result['metadata']['location']['state']}")
    print("  " + "─"*50)

🧪 Testing embedding quality with sample queries...
🎯 Model: all-MiniLM-L6-v2 (384 dimensions)
📊 Testing against 4,160 tourism chunks

--- Query 1: 'adventure activities in Himachal Pradesh mountains' ---
  1. Similarity: 0.7549
     Content: Friendship Peak Expedition – Trek to one of the majestic mountains; Friendship Peak, in the beautifu...
     Category: activities
     Location: Himachal Pradesh, Himachal
  2. Similarity: 0.7492
     Content: If planning for a trek up the beautiful mountains, don't forget to hire a local guide. As the local ...
     Category: activities
     Location: Kullu, Himachal
  3. Similarity: 0.7387
     Content: River rafting, skiing, zorbing, trekking, snow scootering, and river crossing. One of the best adven...
     Category: activities
     Location: Manali, Himachal
  ──────────────────────────────────────────────────

--- Query 2: 'traditional food in Almora' ---
  1. Similarity: 0.5699
     Content: Almora is an agricultural base and also a trade c

In [10]:
# Embedding generation summary
print("\n🎯 EMBEDDING GENERATION COMPLETE!")
print("="*50)
print(f"📊 Model: all-MiniLM-L6-v2 (384 dimensions)")
print(f"📈 Processed: {len(embeddings):,} tourism chunks")
print(f"💾 Output size: {embeddings.nbytes / 1024 / 1024:.1f} MB")
print(f"📁 Files saved: JSON dataset + NumPy vectors")
print(f"🚀 Ready for RAG integration with vector database!")


🎯 EMBEDDING GENERATION COMPLETE!
📊 Model: all-MiniLM-L6-v2 (384 dimensions)
📈 Processed: 4,160 tourism chunks
💾 Output size: 6.1 MB
📁 Files saved: JSON dataset + NumPy vectors
🚀 Ready for RAG integration with vector database!
